# Converting Images into a Region Adjacency Graph
---

## Dependencies
1. **skimage**
  - for creating labelled components from the image (skimage.measure)
  - for finding centroid of each label (skimage.measure)
  - converting the labelled image into a RAG (skimage.future.graph)
  
2. **numpy**
  - image array manipulation operations
  
3. **pickle**
  - dumping the graphs in a file
  
4. **tqdm**
  - for progress bars

5. **cv2**
  - create video files

6. **networkx**
  - for visualizing the graphs
7. **os**
  - saving images and videos
8. **scipy**
  - for developing kdtree to find lenght of shared boundary
  

In [1]:
from skimage import measure, io, draw
from skimage.color import rgb2gray
from skimage.future import graph
import numpy as np
import pickle
from tqdm.notebook import tqdm
import cv2
import networkx as nx
import os
from scipy.spatial import KDTree

### Dev Dependencies

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.set_cmap('gray')
import networkx

<Figure size 432x288 with 0 Axes>

---

## Utility Functions

#### Draw RAG edges and nodes over the image

In [3]:
def draw_edges(img, rag):
    edge_img = img.copy() # to avoid modifying the original image
    for edge in rag.edges:
        # get the node pair for the edge
        node_1, node_2 = edge
        
        # get the cordinated of the centroid of the node
        x1, y1 = map(int, rag.nodes[node_1]['centroid'])
        x2, y2 = map(int, rag.nodes[node_2]['centroid'])
        
        # draw the line joining the centroid of the 2 nodes
        line_x, line_y = draw.line(x1,y1,x2,y2)
        edge_img[line_x, line_y] = 127
 
    return edge_img

In [4]:
def draw_nodes(img, rag):
    node_img = img.copy() # to avoid modifying the original image
    for node in rag.nodes(data=True):
        
        # get the coordinates of the centroid of the node
        node_x, node_y = node[1]['centroid']
        
        # draw the circle at the centroid of the node
        circle_x, circle_y = draw.circle(node_x, node_y, 2)
        node_img[circle_x-1, circle_y-1] = 127
 
    return node_img

#### Save graph overlay images
Create output images containing the original image, labeled_image and graph overlayed over labeled image

In [15]:
def generate_overalay_graph_images(img, label_img, graph_image, img_name, combined=True):
    """
    Style 1 has 3 images stacked over each other. 
    The original image, the label image(where each blob has a different label) 
    and graph overlay image.
    Args:
        img: original grayscale image
        label_img: labelled_image
        graph_img: label_img with graph pverlay on it
        img_name: name of tghe image
        combined: store each trajectory images in a
                  separate folder if False. Default=True.
    Returns:
        Boolean describing the success of the operation
    """
    #combined_img = np.concatenate([img,label_img,graph_img], axis=0)
    
    fig = plt.figure(figsize=(10,10))
    
    # plot the original image
    ax = fig.add_subplot(3, 1, 1)
    ax.imshow(img, cmap="gray")
    ax.set_title("Original Image")
    
    # Plot the labeled Image
    ax = fig.add_subplot(3, 1, 2)
    ax.imshow(label_img, cmap="nipy_spectral")
    ax.set_title("Labelled Image")

    # Plot the Graph Overlay Image
    ax = fig.add_subplot(3, 1, 3)
    ax.imshow(graph_img, cmap="nipy_spectral")
    ax.set_title("Graph Overlay Image")
    
    trajectory_names = {0:"BR0.5-CHI2.4",
                            1:"BR0.5-CHI2.6",
                            2:"BR0.5-CHI2.8",
                            3:"BR0.5-CHI3.0",
                            4:"BR0.52-CHI2.4",
                            5:"BR0.52-CHI2.6",
                            6:"BR0.52-CHI2.8",
                            7:"BR0.52-CHI3.0",
                            8:"BR0.54-CHI2.4",
                            9:"BR0.54-CHI2.6",
                            10:"BR0.54-CHI2.8",
                            11:"BR0.54-CHI3.0",
                            12:"BR0.56-CHI2.4",
                            13:"BR0.56-CHI2.6",
                            14:"BR0.56-CHI2.8",
                            15:"BR0.56-CHI3.0"}
    trajectory_name = trajectory_names[int(img_name)//80]
    title = f"{trajectory_name} #{int(img_name)%80} ({img_name})"
    
    fig.suptitle(title, fontsize=15, y=0.03)    
    plt.tight_layout(rect=[0, 0.05, 1, 1])
    
    
    if combined:
        plt.savefig(os.path.join("../outputs/graph_overlay_combined_images/",img_name+".jpg"))
    else:
        plt.savefig(os.path.join("../outputs/graph_overlay_individual_trajectory_images/",trajectory_name,img_name+".jpg"))
    plt.close()   


#### Save graph videos
Create a video of the saved images

In [21]:
def generate_overalay_graph_video(source_dir, target_dir):
    """
    Find all the images in a given 
    directory and create a video of them.
    Args:
        dir: path to the directory where all the images
             are stored
    """
    # read all the images from the source directory
    source_dir += "/*.jpg" #asuming files are always written in jpg
    imgs = io.imread_collection(source_dir, conserve_memory=True)
    
    # create a video writer in the target location
    height, width, layers = imgs[0].shape
    size = (width,height)
    target_dir += "/graph_overlay.avi"
    out = cv2.VideoWriter(target_dir,cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
    
    # write the frames to the video writer 
    for img in imgs:
        out.write(img)
        out.write(img)
    out.release()
    

#### Save networkx graph visualization images
Graphs created using networkx's draw_networkx function

In [16]:
def generate_graph_visualization_images(graphs, combined=True):
    """
    Save graph visualizations as images.
    Args:
        graphs: list of graphs to be visualized
        combined: if False, directory name will
                  be determined based on index number 
                  of the graph. Default=True.
    """
    
    for graph_num,gg in enumerate(tqdm(graphs)):
        
        fig,axes = plt.subplots()
        
        # setting node size
        node_size = [i[1]['area'] for i in gg.nodes(data=True)]
        sum_node_size = sum(node_size)
        node_size_normalized = [(i/sum_node_size)*5000 for i in node_size]
        
        # setting node color
        node_color = []

        for i in gg.nodes(data=True):
            current_color = i[1]['color']
            if current_color == 1:
                # this is white
                # set to light grey
                node_color.append(np.array([0.7,0.7,0.7]))
            elif current_color == 0:
                # this is black
                # set to dark grey
                node_color.append(np.array([0.3,0.3,0.3]))
            else:
                # this should never happen
                print("Unknown color of node.")
        
        # setting node label
        node_labels = {}
        for index, size in enumerate(node_size):
            node_labels[index+1] = size
            
        # create the graph and save it
        aa = nx.draw_kamada_kawai(gg, 
                             node_size   = node_size_normalized, 
                             node_color  = node_color,
                             edgecolors  = 'k',
                             labels      = node_labels,
                             with_labels = True,
                             ax          = axes)
        
        trajectory_names = {0:"BR0.5-CHI2.4",
                            1:"BR0.5-CHI2.6",
                            2:"BR0.5-CHI2.8",
                            3:"BR0.5-CHI3.0",
                            4:"BR0.52-CHI2.4",
                            5:"BR0.52-CHI2.6",
                            6:"BR0.52-CHI2.8",
                            7:"BR0.52-CHI3.0",
                            8:"BR0.54-CHI2.4",
                            9:"BR0.54-CHI2.6",
                            10:"BR0.54-CHI2.8",
                            11:"BR0.54-CHI3.0",
                            12:"BR0.56-CHI2.4",
                            13:"BR0.56-CHI2.6",
                            14:"BR0.56-CHI2.8",
                            15:"BR0.56-CHI3.0"}
        
        title = trajectory_names[graph_num//80]+f" #{graph_num%80} ({graph_num})"
        plt.title(title, y=-0.1)
        if combined:
            target_dir = f"../outputs/graph_visualizations_combined/{title}.jpg"
        else:
            target_dir = f"../outputs/graph_visualizations_individual_trajectory/{trajectory_names[graph_num//80]}/{title}.jpg"
        plt.savefig(target_dir, format="JPG")
        plt.cla()
        plt.close()
        
    

---

## Generate Region Adjacency Graph (RAG)

#### Steps
1. Read the image, convert to grescale and thresh
2. Assign labels to every black and white region(blob).
3. Generate the Region Adjacency Graph(RAG).
4. Add other attributes to the labels such as node area, color and centroid.
5. Add weight of edges based on length of boundary shared between the 2 regions

*Adaptive thresholding is not a good idea here because we need a constant threshold for all images as we will be calculating similarities in consecutive threshed images*

In [78]:
# read all the images
img_filenames = ["../outputs/images/"+str(i)+".jpg" for i in range(1,1281)]
imgs = io.imread_collection(img_filenames, conserve_memory=True)

graphs = []
kdtree = {}
for index,img in enumerate(tqdm(imgs), start=0):
    
    # convert to grayscale and threshold
    gray_img = rgb2gray(img)
    thresh_img = gray_img > 127

    # generate disinct labels for every region
    # background set to -1 so that black blobs 
    # do not neglected and are assigned a label
    label_img = measure.label(thresh_img, background=-1)

    # generate the Region Adjacency Graph
    rag = graph.RAG(label_img)
    
    # RAG doesn't add a node if there is only 
    # one label in label_image
    if len(np.unique(label_img)) == 1:
        rag.add_node(1)
            
    # add centroid, area and color attributes 
    # to each node using regionprops
    regions = measure.regionprops(label_img, thresh_img)
    for region in regions:
        rag.nodes[region['label']] ['area'] = region['area']
        rag.nodes[region['label']] ['color'] = region['mean_intensity']
        rag.nodes[region['label']] ['centroid'] = region['centroid']
        rag.nodes[region['label']] ['perimeter'] = region['perimeter']
        # will be used to count the number pixels shared on the boundary of 2 regions
        kdtree[region['label']] = KDTree(region.coords)
    
    # add weight of edges
    # weight = number of neghbouring pixels 
    # of the two regions/nodes
    for edge in rag.edges:
        node1, node2 = edge
        rag[node1][node2]['weight'] = kdtree[node1].count_neighbors(kdtree[node2], 1)
    
    
        
    #graph_img = draw_edges(draw_nodes(label_img, rag), rag)
    
    #generate_overalay_graph_images(img, label_img, graph_img, str(index))
    #generate_overalay_graph_images(img, label_img, graph_img, str(index), combined=False)

    graphs.append(rag)

In [77]:
regions[0].perimeter

18.82842712474619

Generate video of all points combined and individual trajectories

In [ ]:
generate_overalay_graph_video("../outputs/graph_overlay_combined_images/", 
                              "../outputs/graph_overlay_combined_videos/")

# write the code for indivisual trajectories if it is needed

Genrate Networkx visualizations for each RAG

In [18]:
generate_graph_visualization_images(graphs)

Dump graph representations to be used elsewhere

In [79]:
pickle.dump(graphs, open('../outputs/pickle/graphs.file', 'wb'))

---